In [2]:
import matplotlib.pyplot as plt
import cv2, re
import numpy as np
from queue import PriorityQueue

In [79]:
import math

def dct(ar):
    n = ar.shape[0]
    d = np.zeros(ar.shape, dtype = 'float32')
    for i in range(8):
        for j in range(8):
            for x in range(8):
                for y in range(8):
                    d[i, j] += math.cos((2 * x + 1) * i * math.pi / 16) * math.cos((2 * y + 1) * j * math.pi / 16) * ar[x, y]
            d[i, j] /= 4
            if (i == 0):
                d[i, j] /= math.sqrt(2)
            if (j == 0):
                d[i, j] /= math.sqrt(2)
    return d

In [165]:
# BMP to JPEG conversion
img = cv2.imread('hello.bmp')

s = img.shape
new_shape = (((s[0] + 15) // 16) * 16, ((s[1] + 15) // 16) * 16, s[2])
new_image = np.zeros(new_shape, dtype = 'uint8')
new_image[:s[0], :s[1], :s[2]] = img
img = np.array(new_image)
old_shape = s
s = new_shape
s1 = (s[0] // 2, s[1] // 2)

# todo - padding

# conversion from RGB format to YCbCr (Y -> Luminance, Cb -> blue chrominance, Cr -> red chrominance)
imgYCC = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
Y = np.array(imgYCC[:,:,0], dtype=int)
Cb = np.zeros(s1, dtype = int)
Cr = np.zeros(s1, dtype = int)

# average every 2x2 block for Cb and Cr
for i in range(s[0]//2):
    for j in range(s[1]//2):
        x = i*2
        y = j*2
        avg = (int(imgYCC[x, y, 1]) + int(imgYCC[x, y + 1, 1]) + int(imgYCC[x + 1, y, 1]) + int(imgYCC[x + 1, y + 1, 1])) // 4
        Cr[i, j] = avg
        avg = (int(imgYCC[x][y][2]) + int(imgYCC[x][y + 1][2]) + int(imgYCC[x + 1][y][2]) + int(imgYCC[x + 1][y + 1][2])) // 4
        Cb[i, j] = avg

# Discrete Cosine Transform
Y -= 128
Cb -= 128
Cr -= 128
Y_blocks = []
Cb_blocks = []
Cr_blocks = []
Y_rgb = []
Cb_rgb = []
Cr_rgb = []
for i in range(0, s1[0], 8):
#     for j in range(0, s1[1], 8):
#         grid = np.float32(Cb[i:i+8, j:j+8])
#         Cb_blocks.append(cv2.dct(grid))
#         grid = np.float32(Cr[i:i+8, j:j+8])
#         Cr_blocks.append(cv2.dct(grid))
#         grid = np.float32(Y[i*2:i*2+8, j*2:j*2+8])
#         Y_blocks.append(cv2.dct(grid))
#         grid = np.float32(Y[i*2:i*2+8, j*2+8:j*2+16])
#         Y_blocks.append(cv2.dct(grid))
#         grid = np.float32(Y[i*2+8:i*2+16, j*2:j*2+8])
#         Y_blocks.append(cv2.dct(grid))
#         grid = np.float32(Y[i*2+8:i*2+16, j*2+8:j*2+16])
#         Y_blocks.append(cv2.dct(grid))
    if False:
        count = 0
        for j in range(0, s1[1], 8):
            count+=1
            grid = np.float32(Cb[i:i+8, j:j+8])
            Cb_rgb.append(grid)
            Cb_blocks.append(cv2.dct(grid))
            
            grid = np.float32(Cr[i:i+8, j:j+8])
            Cr_rgb.append(grid)
            Cr_blocks.append(cv2.dct(grid))
            
            grid = np.float32(Y[i*2:i*2+8, j*2:j*2+8])
            Y_rgb.append(grid)
            Y_blocks.append(cv2.dct(grid))
            
            grid = np.float32(Y[i*2:i*2+8, j*2+8:j*2+16])
            Y_rgb.append(grid)
            Y_blocks.append(cv2.dct(grid))
            
            grid = np.float32(Y[i*2+8:i*2+16, j*2:j*2+8])
            Y_rgb.append(grid)
            Y_blocks.append(cv2.dct(grid))
            
            grid = np.float32(Y[i*2+8:i*2+16, j*2+8:j*2+16])
            Y_rgb.append(grid)
#             if count<-5:
#                 grid = np.zeros((8,8), dtype=float)-128
            Y_blocks.append(cv2.dct(grid))

    else:
        for j in range(0, s1[1], 8):
            grid = np.float32(Cb[i:i+8, j:j+8])
            Cb_blocks.append(cv2.dct(grid))
            grid = np.float32(Cr[i:i+8, j:j+8])
            Cr_blocks.append(cv2.dct(grid))
            grid = np.float32(Y[i*2:i*2+8, j*2:j*2+8])
#             grid = np.zeros((8,8), dtype=float)-128
            Y_rgb.append(grid)
            Y_blocks.append(cv2.dct(grid))
            grid = np.float32(Y[i*2:i*2+8, j*2+8:j*2+16])
            Y_rgb.append(grid)
#             grid = np.zeros((8,8), dtype=float)-128
            Y_blocks.append(cv2.dct(grid))
            grid = np.float32(Y[i*2+8:i*2+16, j*2:j*2+8])
            Y_rgb.append(grid)
#             grid = np.zeros((8,8), dtype=float)-128
            Y_blocks.append(cv2.dct(grid))
            grid = np.float32(Y[i*2+8:i*2+16, j*2+8:j*2+16])
            Y_rgb.append(grid)
#             grid = np.zeros((8,8), dtype=float)-128
            Y_blocks.append(cv2.dct(grid))

len_y = len(Y_blocks)
len_c = len(Cb_blocks)

# Quantization

def spiralo(block):
    p = 8
    arr = []
    k = 0
    for c in range(p):
        for i in range(0, k+1):
            if k%2!=0 :
                arr.append(block[i][k-i])
            else :
                arr.append(block[k-i][i])
        k+=1

    k = 1
    for c in range(p-1):
        j = p
        for i in range(k, p):
            j-=1
            if k%2 == 0:
                arr.append(block[i][j])
            else:
                arr.append(block[j][i])
        k+=1
    return arr

# Quantization
##for snail
myArr = [
[0x03, 0x02, 0x02, 0x03, 0x02, 0x02, 0x03, 0x03],
[0x03, 0x03, 0x04, 0x03, 0x03, 0x04, 0x05, 0x08],
[0x05, 0x05, 0x04, 0x04, 0x05, 0x0a, 0x07, 0x07],
[0x06, 0x08, 0x0c, 0x0a, 0x0c, 0x0c, 0x0b, 0x0a],
[0x0b, 0x0b, 0x0d, 0x0e, 0x12, 0x10, 0x0d, 0x0e],
[0x11, 0x0e, 0x0b, 0x0b, 0x10, 0x16, 0x10, 0x11],
[0x13, 0x14, 0x15, 0x15, 0x15, 0x0c, 0x0f, 0x17],
[0x18, 0x16, 0x14, 0x18, 0x12, 0x14, 0x15, 0x14]
]
##for bmp
myArr = [
    [0x03, 0x02, 0x02, 0x03, 0x02, 0x02, 0x03, 0x03], 
    [0x03, 0x03, 0x04, 0x03, 0x03, 0x04, 0x05, 0x08], 
    [0x05, 0x05, 0x04, 0x04, 0x05, 0x0a, 0x07, 0x07], 
    [0x06, 0x08, 0x0c, 0x0a, 0x0c, 0x0c, 0x0b, 0x0a], 
    [0x0b, 0x0b, 0x0d, 0x0e, 0x12, 0x10, 0x0d, 0x0e], 
    [0x11, 0x0e, 0x0b, 0x0b, 0x10, 0x16, 0x10, 0x11], 
    [0x13, 0x14, 0x15, 0x15, 0x15, 0x0c, 0x0f, 0x17], 
    [0x18, 0x16, 0x14, 0x18, 0x12, 0x14, 0x15, 0x14]]

##for smail
myArr2 = [
[0x03, 0x04, 0x04, 0x05, 0x04, 0x05, 0x09, 0x05],
[0x05, 0x09, 0x14, 0x0d, 0x0b, 0x0d, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14]
]

##for bmp
myArr2 = [ 
    [0x03, 0x04, 0x04, 0x05, 0x04, 0x05, 0x09, 0x05], 
    [0x05, 0x09, 0x14, 0x0d, 0x0b, 0x0d, 0x14, 0x14], 
    [0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14], 
    [0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14], 
    [0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14], 
    [0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14], 
    [0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14], 
    [0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14]]

spiral = [[(i, j) for j in range(8)] for i in range(8)]
spiral = spiralo(spiral)
Y_q = np.array([[4., 3, 4, 4, 4, 6, 11, 15], [3, 3, 3, 4, 5, 8, 14, 19], [3, 4, 4, 5, 8, 12, 16, 20], [4, 5, 6, 7, 12, 14, 18, 20], [6, 6, 9, 11, 14, 17, 21, 23], [9, 12, 12, 18, 23, 22, 25, 21], [11, 13, 15, 17, 21, 23, 25, 21], [13, 12, 12, 13, 16, 19, 21, 21]])
C_q = np.array([[4., 4, 6, 10, 21, 21, 21, 21], [4, 5, 6, 21, 21, 21, 21, 21], [6, 6, 12, 21, 21, 21, 21, 21], [10, 14, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21]])
cur = 0
for i in range(8):
    for j in range(8):
        Y_q[spiral[cur][0]][spiral[cur][1]] = myArr[i][j]
        C_q[spiral[cur][0]][spiral[cur][1]] = myArr2[i][j]
        cur += 1
def round_values(ar):
    ans = np.zeros(ar.shape, dtype = 'float32')
    for i in range(len(ar)):
        for j in range(len(ar[0])):
            ans[i][j] = round(ar[i][j])
    return ans


Y_blocks_og = list(Y_blocks)
Cb_blocks_og = list(Cb_blocks)
Cr_blocks_og = list(Cr_blocks)

for i in range(len_y):
    Y_blocks[i] = round_values(Y_blocks[i] / Y_q)
for i in range(len_c):
    Cb_blocks[i] = round_values(Cb_blocks[i] / C_q)
for i in range(len_c):
    Cr_blocks[i] = round_values(Cr_blocks[i] / C_q)

# Run length encoding
def spiral_traversal(block):
    p = 8
    arr = np.array([])
    k = 0
    for c in range(p):
        for i in range(0, k+1):
            if k%2!=0 :
                arr = np.append(arr, block[i][k-i])
            else :
                arr = np.append(arr, block[k-i][i])
        k+=1

    k = 1
    for c in range(p-1):
        j = p
        for i in range(k, p):
            j-=1
            if k%2 == 0:
                arr = np.append(arr, block[i][j])
            else:
                arr = np.append(arr, block[j][i])
        k+=1
    return arr

dc = 0
def encode(block):
    global dc
    temp = spiral_traversal(block)
    l = len(temp)

    encoded_block = []
    encoded_block.append((0, temp[0] - dc))
    dc = temp[0]
    c0 = 0
    cnz = 0
    for i in range(1, l):
        if temp[i]: cnz += 1
    loc = -1
    for i in range(1, l):
        if cnz == 0:
            loc = i
            break
        if temp[i] or c0==15 :
            encoded_block.append((c0, temp[i]))
            c0 = 0
            if temp[i]:
                cnz -= 1
        else:
            c0 += 1
    
    if loc >-1 :
        encoded_block.append((0, 0))
    
    encoded_block = np.array(encoded_block, dtype = int)
    return encoded_block

encoded_Y = list()
encoded_Cr = list()
encoded_Cb = list()
for i in range(len_y):
    encoded_Y.append(encode(Y_blocks[i]))
dc = 0
for i in range(len_c):
    encoded_Cb.append(encode(Cb_blocks[i]))
dc = 0
for i in range(len_c):
    encoded_Cr.append(encode(Cr_blocks[i]))
encoded_Y = np.array(encoded_Y, dtype = object)
encoded_Cr = np.array(encoded_Cr, dtype = object)
encoded_Cb = np.array(encoded_Cb, dtype = object)

# Huffman Tables
def get_cat(num): ##extracting position of msb to determine what length of bits it will need to be encoded
    num = int(abs(num))
    ans = 0
    pwr = 1
    while pwr <= num:
        pwr <<= 1
        ans += 1
    return ans

def make_freq_table(dc_freq, ac_freq, encoded_blocks):
    for block in encoded_blocks:
        #block of length 64 usually (we take 8*8 grids), first element is dc coeff, the rest are ac coeffs
        #dc block[0] is encoded as (0, value)
        dc = block[0]
        dc_run_length, dc_val = dc #dc_run_length will be 0
        
        cat = get_cat(dc_val)
        dc_freq[dc_run_length, cat] += 1
        
        for i in range(1, len(block)):
            
            #ac block[i] is encoded as (run length of zeros before value max 15, value of ith non zero ac coefficient)
            ac_i = block[i]
            run_length, value = ac_i
            cat = get_cat(value) # find the category of value
            ac_freq[run_length, cat] += 1
            
dc_huffman_Y = np.zeros((1, 16), dtype=int)
ac_huffman_Y = np.zeros((16,16), dtype=int)
dc_huffman_C = np.zeros((1, 16), dtype=int)
ac_huffman_C = np.zeros((16,16), dtype=int)

make_freq_table(dc_huffman_Y, ac_huffman_Y, encoded_Y)
make_freq_table(dc_huffman_C, ac_huffman_C, encoded_Cr)
make_freq_table(dc_huffman_C, ac_huffman_C, encoded_Cb)



# make huffman codes

hufs = [[ 0x00, 0x01, 0x05, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 
         0x00, 0x00, 0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0a, 0x0b],
        [ 0x00, 0x02, 0x01, 0x03, 0x03, 0x02, 0x04, 0x03, 0x05, 0x05, 0x04, 0x04, 0x00, 
         0x00, 0x01, 0x7d, 0x01, 0x02, 0x03, 0x00, 0x04, 0x11, 0x05, 0x12, 0x21, 0x31, 0x41,
         0x06, 0x13, 0x51, 0x61, 0x07, 0x22, 0x71, 0x14, 0x32, 0x81, 0x91, 0xa1, 0x08, 0x23, 
         0x42, 0xb1, 0xc1, 0x15, 0x52, 0xd1, 0xf0, 0x24, 0x33, 0x62, 0x72, 0x82, 0x09, 0x0a, 
         0x16, 0x17, 0x18, 0x19, 0x1a, 0x25, 0x26, 0x27, 0x28, 0x29, 0x2a, 0x34, 0x35, 0x36, 
         0x37, 0x38, 0x39, 0x3a, 0x43, 0x44, 0x45, 0x46, 0x47, 0x48, 0x49, 0x4a, 0x53, 0x54, 
         0x55, 0x56, 0x57, 0x58, 0x59, 0x5a, 0x63, 0x64, 0x65, 0x66, 0x67, 0x68, 0x69, 0x6a, 
         0x73, 0x74, 0x75, 0x76, 0x77, 0x78, 0x79, 0x7a, 0x83, 0x84, 0x85, 0x86, 0x87, 0x88, 
         0x89, 0x8a, 0x92, 0x93, 0x94, 0x95, 0x96, 0x97, 0x98, 0x99, 0x9a, 0xa2, 0xa3, 0xa4, 
         0xa5, 0xa6, 0xa7, 0xa8, 0xa9, 0xaa, 0xb2, 0xb3, 0xb4, 0xb5, 0xb6, 0xb7, 0xb8, 0xb9, 
         0xba, 0xc2, 0xc3, 0xc4, 0xc5, 0xc6, 0xc7, 0xc8, 0xc9, 0xca, 0xd2, 0xd3, 0xd4, 0xd5, 
         0xd6, 0xd7, 0xd8, 0xd9, 0xda, 0xe1, 0xe2, 0xe3, 0xe4, 0xe5, 0xe6, 0xe7, 0xe8, 0xe9, 
         0xea, 0xf1, 0xf2, 0xf3, 0xf4, 0xf5, 0xf6, 0xf7, 0xf8, 0xf9, 0xfa],
        [ 0x00, 0x03, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x00, 0x00, 0x00, 
         0x00, 0x00, 0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0a, 0x0b],
        [ 0x00, 0x02, 0x01, 0x02, 0x04, 0x04, 0x03, 0x04, 0x07, 0x05, 0x04, 0x04, 0x00, 
         0x01, 0x02, 0x77, 0x00, 0x01, 0x02, 0x03, 0x11, 0x04, 0x05, 0x21, 0x31, 0x06, 0x12, 
         0x41, 0x51, 0x07, 0x61, 0x71, 0x13, 0x22, 0x32, 0x81, 0x08, 0x14, 0x42, 0x91, 0xa1, 
         0xb1, 0xc1, 0x09, 0x23, 0x33, 0x52, 0xf0, 0x15, 0x62, 0x72, 0xd1, 0x0a, 0x16, 0x24, 
         0x34, 0xe1, 0x25, 0xf1, 0x17, 0x18, 0x19, 0x1a, 0x26, 0x27, 0x28, 0x29, 0x2a, 0x35, 
         0x36, 0x37, 0x38, 0x39, 0x3a, 0x43, 0x44, 0x45, 0x46, 0x47, 0x48, 0x49, 0x4a, 0x53, 
         0x54, 0x55, 0x56, 0x57, 0x58, 0x59, 0x5a, 0x63, 0x64, 0x65, 0x66, 0x67, 0x68, 0x69, 
         0x6a, 0x73, 0x74, 0x75, 0x76, 0x77, 0x78, 0x79, 0x7a, 0x82, 0x83, 0x84, 0x85, 0x86, 
         0x87, 0x88, 0x89, 0x8a, 0x92, 0x93, 0x94, 0x95, 0x96, 0x97, 0x98, 0x99, 0x9a, 0xa2, 
         0xa3, 0xa4, 0xa5, 0xa6, 0xa7, 0xa8, 0xa9, 0xaa, 0xb2, 0xb3, 0xb4, 0xb5, 0xb6, 0xb7, 
         0xb8, 0xb9, 0xba, 0xc2, 0xc3, 0xc4, 0xc5, 0xc6, 0xc7, 0xc8, 0xc9, 0xca, 0xd2, 0xd3, 
         0xd4, 0xd5, 0xd6, 0xd7, 0xd8, 0xd9, 0xda, 0xe2, 0xe3, 0xe4, 0xe5, 0xe6, 0xe7, 0xe8, 
         0xe9, 0xea, 0xf2, 0xf3, 0xf4, 0xf5, 0xf6, 0xf7, 0xf8, 0xf9, 0xfa]]


depth = 0
def get_codebook(freq, index):
    codebook = np.empty(freq.shape, dtype = object)
    def get_code_lengths_from_huf(huf_table, m, n):
        #for dc m = 1, n = 16
        code_lengths = np.zeros((m,n), dtype=int)
        counts = np.zeros(17, dtype=int)
        for length in range(1, 17):
            counts[length] = huf_table[length-1]
        l = len(huf_table)
        ctr = 0
        for w in range(16, l):
            symbol = huf_table[w]
            i = symbol//16
            j = symbol%16
            while counts[ctr]==0: ctr+=1
            code_lengths[i, j] = ctr
            counts[ctr]-=1
        return code_lengths
    
    def get_code_lengths(freq):

        def make_tree(freq):
            global code
            q = PriorityQueue()
            m = len(freq)
            n = len(freq[0])

            cnt = 0
            for i in range(m):
                for j in range(n):
                    if freq[i,j]:
                        q.put((freq[i,j], str((i,j))))
                        cnt += 1

            for i in range(cnt-1):
                a = q.get()
                b = q.get()
                q.put((a[0]+b[0], str("[" + a[1] + "," + b[1] + "]")))

            tree = eval(q.get()[1])
            return tree

        tree = make_tree(freq)
        code_lengths = np.zeros(freq.shape, dtype=int)

        depth = 0
        def get_depths(arr, code_lengths):
            global depth
            if type(arr[1]) == int and type(arr[0]) == int:
                code_lengths[arr[0], arr[1]] = min(depth, 16)
            else:
                depth += 1
                get_depths(arr[0], code_lengths)
                get_depths(arr[1], code_lengths)
                depth-=1

        get_depths(tree, code_lengths)
        return code_lengths
    
    code_lengths = get_code_lengths_from_huf(hufs[index], freq.shape[0], freq.shape[1])
    print(code_lengths)
#     code_lengths = get_code_lengths(freq)
    print(get_code_lengths(freq))
    print()
    
    length_symbol_pairs = []
    
    m, n = freq.shape
    for i in range(m):
        for j in range(n):
            if code_lengths[i, j]:
                length_symbol_pairs.append((code_lengths[i, j], (i, j)))
    length_symbol_pairs.sort()
    
    def get_code(code, length):
        st = bin(code).replace("0b", "")[::-1]
        while len(st) < length:
            st += '0'
        return st[::-1]
    
    code = 0
    cur_len = 0
    for length, symbol in length_symbol_pairs:
        while cur_len < length:  #if there is no symbol with the current length
            cur_len += 1
            code *= 2
        codebook[symbol[0], symbol[1]] = get_code(code, length)
        code += 1
    
    return codebook, length_symbol_pairs

codes_dc_Y, sorted_symbols_dc_Y = get_codebook(dc_huffman_Y, 0)
codes_ac_Y, sorted_symbols_ac_Y = get_codebook(ac_huffman_Y, 1)
codes_dc_C, sorted_symbols_dc_C = get_codebook(dc_huffman_C, 2)
codes_ac_C, sorted_symbols_ac_C = get_codebook(ac_huffman_C, 3)

def to_two_bytes(num):
    bh = bin(num).replace("0b", "")
    bh = bh[::-1]
    while len(bh) < 16:
        bh += '0'
    bh = bh[::-1]

    return [int(bh[:8], 2), int(bh[8:], 2)]

# Writing jpeg image in binary
jpeg_image = [0xff, 0xd8, 0xff, 0xe0, 0x00, 0x10]

header = [0x4a, 0x46, 0x49, 0x46, 0x00, 0x01, 0x01, 0x01, 0x00, 0x60, 0x00, 0x60, 0x00, 0x00]
jpeg_image.extend(header)

luminance_quantisation_table = [0xff, 0xdb, 0x00, 0x43, 0x00]
luminance_quantisation_table.extend([int(ele) for ele in spiral_traversal(Y_q)])
jpeg_image.extend(luminance_quantisation_table)

chrominance_quantisation_table = [0xff, 0xdb, 0x00, 0x43, 0x01]
chrominance_quantisation_table.extend([int(ele) for ele in spiral_traversal(C_q)])
jpeg_image.extend(chrominance_quantisation_table)

start_of_frame = [0xff, 0xc0, 0x00, 0x11, 0x08]
start_of_frame.extend(to_two_bytes(old_shape[0]))
start_of_frame.extend(to_two_bytes(old_shape[1]))
start_of_frame.extend([0x03, 0x01, 0x22, 0x00, 0x02, 0x11, 0x01, 0x03, 0x11, 0x01])
jpeg_image.extend(start_of_frame)

def parse_table(sorted_symbols, typ):
    table = [0xff, 0xc4]
    ln = 19
    symbols = []
    freqs = [0 for i in range(16)]
    
    for length, symbol in sorted_symbols:
        freqs[length-1] += 1
        symbols.append(symbol[0] * 16 + symbol[1])
        ln += 1
    
    table.extend(to_two_bytes(ln))
    table.append(typ)
    table.extend(freqs)
    table.extend(symbols)
    return table

jpeg_image.extend(parse_table(sorted_symbols_dc_Y, 0x00))
jpeg_image.extend(parse_table(sorted_symbols_ac_Y, 0x10))
jpeg_image.extend(parse_table(sorted_symbols_dc_C, 0x01))
jpeg_image.extend(parse_table(sorted_symbols_ac_C, 0x11))

start_of_scan = [0xff, 0xda, 0x00, 0x0c, 0x03, 0x01, 0x00, 0x02, 0x11, 0x03, 0x11, 0x00, 0x3f, 0x00]
jpeg_image.extend(start_of_scan)

def bit_rep(value):
    if value == 0:
        return ''
    else : 
        v = bin(abs(value)).replace("0b", '')
        if value>0 :
            return v
        else:
            return v.replace('0', 'a').replace('1','0').replace('a','1')

def encode_data(encoded_Y, encoded_Cb, encoded_Cr, codes_dc_Y, code_dc_C, codes_ac_Y, codes_ac_C):
    s = ''
    def encode_block(block, dc_codebook, ac_codebook, flag, temp):
        print(block)
        ans = ''
        dc_val = block[0][1]
        cat = get_cat(dc_val)
#         print(dc_codebook[0,cat]+bit_rep(dc_val))
        ans+=(dc_codebook[0,cat]+bit_rep(dc_val))
        
        for i in range(1, len(block)):
            run, val = block[i]
#             print(ac_codebook[run, get_cat(val)]+bit_rep(val))
            ans+=(ac_codebook[run, get_cat(val)]+bit_rep(val))
        return ans
    print(len(encoded_Y), len(encoded_Cb))
    for i in range(len(encoded_Y)):
        print(i)
        s += encode_block(encoded_Y[i], codes_dc_Y, codes_ac_Y, 0, i)
        if (i% 4 == 3):
            print('cbcr')
            s += encode_block(encoded_Cb[(i) // 4], codes_dc_C, codes_ac_C, 1, i//4)
            s += encode_block(encoded_Cr[(i) // 4], codes_dc_C, codes_ac_C, 2, i//4)
        
    #pad with 0s
    a = len(s)%8
    s+='0'*a
    
    data = [int(s[i:i+8], 2) for i in range(0, len(s), 8)]
    fin_data = []
    for ele in data:
        fin_data.append(ele)
        if ele == 0xff:
            fin_data.append(0x00)
    return fin_data

jpeg_image.extend(encode_data(encoded_Y, encoded_Cb, encoded_Cr, codes_dc_Y, codes_dc_C, codes_ac_Y, codes_ac_C))

end_of_image = [0xff, 0xd9]
jpeg_image.extend(end_of_image)

try:
    with open("my_image.jpeg", 'wb') as f:
        for byte in jpeg_image:
            f.write(byte.to_bytes(1, byteorder='big'))
except Exception as e:
    print(e)

file = open('my_image.jpeg', 'rb')
data = file.read()
s = data.hex()

l = re.findall('..?', s)
while len(l) % 11:
    l = np.append(l, '0')
d = np.reshape(l, (-1,11))
d = [' '.join(ele) for ele in d]
print(len(d))
# for ele in d:
#     print(ele)

file.close()

[[2 3 3 3 3 3 4 5 6 7 8 9 0 0 0 0]]
[[5 4 4 2 2 3 3 4 6 6 0 0 0 0 0 0]]

[[ 4  2  2  3  4  5  7  8 10 16 16  0  0  0  0  0]
 [ 0  4  5  7  9 11 16 16 16 16 16  0  0  0  0  0]
 [ 0  5  8 10 12 16 16 16 16 16 16  0  0  0  0  0]
 [ 0  6  9 12 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0  6 10 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0  7 11 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0  7 12 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0  8 12 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0  9 15 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0  9 16 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0  9 16 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0 10 16 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0 10 16 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0 11 16 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0 16 16 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [11 16 16 16 16 16 16 16 16 16 16  0  0  0  0  0]]
[[ 3  2  3  3  4  5  6  8  9  0  0  0  0  0  0  0]
 [ 0  4  5  6  7 10 11 13 11  0  0  0  0  0  0  0]
 [ 0  5 

 [ 0  0]]
cbcr
[[ 0 -2]
 [ 1  2]
 [ 0  0]]
[[ 0  3]
 [ 1 -1]
 [ 0  0]]
364
[[  0 -16]
 [  0   1]
 [  0  -7]
 [  1   1]
 [  4  -1]
 [  0   0]]
365
[[ 0  0]
 [ 1 -6]
 [ 0  0]]
366
[[ 0 14]
 [ 0  1]
 [ 0 -7]
 [ 6 -1]
 [ 0  0]]
367
[[ 0 -1]
 [ 0  1]
 [ 0 -6]
 [ 0  1]
 [ 0  0]]
cbcr
[[ 0 -1]
 [ 0  1]
 [ 0  2]
 [ 0  0]]
[[ 0  3]
 [ 0 -1]
 [ 0 -1]
 [ 0  0]]
368
[[  0 -14]
 [  0  -1]
 [  0  -6]
 [  0   0]]
369
[[ 0  0]
 [ 1 -6]
 [ 2  2]
 [ 0  0]]
370
[[ 0 14]
 [ 1 -6]
 [ 0  0]]
371
[[ 0  0]
 [ 0 -1]
 [ 0 -5]
 [ 2  1]
 [ 0  0]]
cbcr
[[ 0 -4]
 [ 0  1]
 [ 0  2]
 [ 0  0]]
[[ 0  4]
 [ 0 -1]
 [ 0 -2]
 [ 0 -1]
 [ 0  0]]
372
[[  0 -14]
 [  0  -1]
 [  0  -7]
 [  0   0]]
373
[[ 0  0]
 [ 1 -5]
 [ 0  0]]
374
[[ 0 13]
 [ 0  1]
 [ 0 -6]
 [ 0  0]]
375
[[ 0  0]
 [ 1 -5]
 [ 2  2]
 [ 1 -1]
 [ 1 -1]
 [ 0  0]]
cbcr
[[ 0 -3]
 [ 0  1]
 [ 0  3]
 [ 0  0]]
[[ 0  4]
 [ 0 -1]
 [ 0 -1]
 [ 0  0]]
376
[[  0 -13]
 [  0   1]
 [  0  -5]
 [  6  -1]
 [  0   0]]
377
[[ 0  1]
 [ 1 -3]
 [ 0  0]]
378
[[ 0 11]
 [ 1 -6]
 [ 3  1]
 [ 0

 [ 0  0]]
746
[[ 0 -5]
 [ 0 -3]
 [ 0  0]]
747
[[ 0  6]
 [ 0 -2]
 [ 0  0]]
cbcr
[[ 0 -6]
 [ 0  2]
 [ 0  0]]
[[ 0 -1]
 [ 1 -1]
 [ 0  0]]
748
[[ 0  7]
 [ 0 -3]
 [ 0  0]]
749
[[ 0  5]
 [ 0 -3]
 [ 0 -1]
 [ 0  0]]
750
[[ 0 -5]
 [ 0 -3]
 [ 0 -1]
 [ 1 -1]
 [ 0  0]]
751
[[ 0  7]
 [ 0 -3]
 [ 0  0]]
cbcr
[[ 0 -7]
 [ 0  2]
 [ 0  0]]
[[ 0 -1]
 [ 0  0]]
752
[[ 0  5]
 [ 0 -2]
 [ 0 -1]
 [ 3 -1]
 [ 0  0]]
753
[[ 0  6]
 [ 0 -2]
 [ 3 -1]
 [ 0  0]]
754
[[ 0 -5]
 [ 0 -3]
 [ 3  1]
 [ 0  0]]
755
[[ 0  6]
 [ 0 -1]
 [ 3 -2]
 [ 0  0]]
cbcr
[[ 0 -7]
 [ 0  1]
 [ 0  0]]
[[ 0 -2]
 [ 0  1]
 [ 0 -1]
 [ 0  0]]
756
[[ 0  4]
 [ 0 -2]
 [ 0  1]
 [ 0  0]]
757
[[ 0  4]
 [ 0 -2]
 [ 0 -1]
 [ 0  0]]
758
[[ 0 -4]
 [ 0 -3]
 [ 0 -1]
 [ 0  0]]
759
[[ 0  7]
 [ 0 -2]
 [ 0 -1]
 [ 0  0]]
cbcr
[[ 0 -6]
 [ 0  2]
 [ 0  0]]
[[ 0 -4]
 [ 0  0]]
760
[[ 0  1]
 [ 0 -2]
 [ 0 -1]
 [ 0 -1]
 [ 0  0]]
761
[[ 0  4]
 [ 0 -2]
 [ 0 -2]
 [ 0  0]]
762
[[ 0  1]
 [ 0 -3]
 [ 0 -3]
 [ 0  0]]
763
[[ 0  6]
 [ 0 -2]
 [ 0 -2]
 [ 0  0]]
cbcr
[[ 0 -5]
 [ 0  1]
 [ 

[[ 0 -7]
 [ 0  1]
 [ 0  0]]
1092
[[ 0 -6]
 [ 0  3]
 [ 0 -3]
 [ 2 -1]
 [ 0  1]
 [ 0  0]]
1093
[[ 0 -2]
 [ 0 -2]
 [ 0 -1]
 [ 0  0]]
1094
[[ 0  8]
 [ 1 -1]
 [ 0  1]
 [ 0  0]]
1095
[[ 0 -2]
 [ 0  1]
 [ 0 -1]
 [ 0  0]]
cbcr
[[ 0  6]
 [ 0 -2]
 [ 0 -2]
 [ 0 -1]
 [ 1 -1]
 [ 0  0]]
[[ 0 -3]
 [ 0  0]]
1096
[[ 0 -8]
 [ 0  1]
 [ 0 -3]
 [ 1 -2]
 [ 0  0]]
1097
[[ 0 -3]
 [ 0  2]
 [ 0 -1]
 [ 0  0]]
1098
[[ 0  9]
 [ 0  1]
 [ 0 -2]
 [ 0  0]]
1099
[[ 0 -4]
 [ 0  1]
 [ 0 -1]
 [ 0  0]]
cbcr
[[ 0  4]
 [ 0 -1]
 [ 0 -1]
 [ 1  1]
 [ 0 -1]
 [ 0  0]]
[[ 0 -2]
 [ 0  1]
 [ 0  0]]
1100
[[  0 -10]
 [  0   2]
 [  0  -3]
 [  0   0]]
1101
[[ 0 -5]
 [ 0  3]
 [ 0 -2]
 [ 6 -1]
 [ 0  0]]
1102
[[ 0 10]
 [ 0  2]
 [ 0 -1]
 [ 2 -1]
 [ 0  1]
 [ 0  0]]
1103
[[ 0 -7]
 [ 0  3]
 [ 0 -2]
 [ 0  0]]
cbcr
[[ 0  3]
 [ 0 -1]
 [ 0 -2]
 [ 0  0]]
[[ 0 -2]
 [ 2  1]
 [ 0  0]]
1104
[[  0 -10]
 [  0   2]
 [  0  -3]
 [  0   0]]
1105
[[ 0 -8]
 [ 0  4]
 [ 0 -3]
 [ 0 -1]
 [ 0  0]]
1106
[[ 0 11]
 [ 0  3]
 [ 0 -1]
 [ 2 -2]
 [ 0  0]]
1107
[[ 0 -7]
 [ 

1485
[[  0  -2]
 [  0   1]
 [  0 -10]
 [  0  -1]
 [  0   0]]
1486
[[ 0 26]
 [ 0  1]
 [ 0 -9]
 [ 0 -1]
 [ 5 -1]
 [ 0  0]]
1487
[[  0  -3]
 [  1 -10]
 [  1   1]
 [  2  -1]
 [  1  -1]
 [  0   0]]
cbcr
[[  0 -13]
 [  0   3]
 [  0   8]
 [  1  -1]
 [  0   0]]
[[ 0  6]
 [ 0 -1]
 [ 0  3]
 [ 0  0]]
1488
[[  0 -26]
 [  0   1]
 [  0 -11]
 [  0  -1]
 [  5  -1]
 [  0   0]]
1489
[[  0   0]
 [  1 -13]
 [  1  -1]
 [  0   0]]
1490
[[  0  26]
 [  1 -11]
 [  3  -1]
 [  0   0]]
1491
[[  0  -1]
 [  0   1]
 [  0 -11]
 [  4  -2]
 [  1  -1]
 [  1  -2]
 [  0   0]]
cbcr
[[  0 -14]
 [  0   2]
 [  0  10]
 [  2   1]
 [  0   0]]
[[ 0  4]
 [ 1  4]
 [ 1 -1]
 [ 0  0]]
1492
[[  0 -26]
 [  1 -11]
 [  4  -1]
 [  0   0]]
1493
[[  0  -5]
 [  0   3]
 [  0 -11]
 [  2  -2]
 [  0   1]
 [  0   0]]
1494
[[  0  29]
 [  0   1]
 [  0 -11]
 [  2  -1]
 [  0  -1]
 [  0   0]]
1495
[[  0  -5]
 [  0   4]
 [  0 -10]
 [  6  -1]
 [  0   0]]
cbcr
[[ 0 -8]
 [ 0  2]
 [ 0  9]
 [ 0 -1]
 [ 0  0]]
[[ 0  5]
 [ 0 -1]
 [ 0  5]
 [ 0  0]]
1496
[[  0 -2

...

cbcr
[[  0 -32]
 [  0   4]
 [  0  12]
 [  0  -1]
 [  0   1]
 [  0  -1]
 [  0   1]
 [  0   2]
 [  1   1]
 [  1  -1]
 [  0  -3]
 [  0  -1]
 [  4   1]
 [  4   1]
 [  0   1]
 [  7  -1]
 [  0   0]]
[[ 0  9]
 [ 0  1]
 [ 0 -1]
 [ 0  1]
 [ 1  1]
 [ 1 -2]
 [ 0 -1]
 [ 3  1]
 [ 0  0]]
3068
[[  0 -22]
 [  0   6]
 [  0 -38]
 [  0 -42]
 [  0 -12]
 [  0  -6]
 [  0   1]
 [  0  -4]
 [  0   7]
 [  2   2]
 [  0   6]
 [  3   1]
 [  0   2]
 [  0   1]
 [  0  -9]
 [  0  -1]
 [  1  -1]
 [  0  -1]
 [  0  -6]
 [  7   1]
 [  0   1]
 [  0  -1]
 [  0  -1]
 [  0   1]
 [  0  -1]
 [  0  -1]
 [  7  -1]
 [  0   1]
 [  0   0]]
3069
[[  0 -21]
 [  0 -18]
 [  0 -32]
 [  0 -28]
 [  0  -7]
 [  0  -2]
 [  0   3]
 [  0   5]
 [  0  16]
 [  0  24]
 [  0   1]
 [  0 -10]
 [  1  -1]
 [  0   1]
 [  2  -2]
 [  0  -6]
 [  0  -9]
 [  0  -7]
 [  0   1]
 [  0   5]
 [  0   5]
 [  0   3]
 [ 12  -1]
 [  0  -1]
 [  0   0]]
3070
[[ 0 33]
 [ 0 18]
 [ 0  4]
 [ 0  4]
 [ 0 12]
 [ 1 -4]
 [ 0 -9]
 [ 0 -9]
 [ 0 -1]
 [ 1 -3]
 [ 0 -8]
 [ 0 -1]
 [ 3 -

 [  0   0]]
3297
[[   0 -104]
 [   0    5]
 [   0  -30]
 [   0    2]
 [   0  -11]
 [   0    3]
 [   1   -3]
 [   0    1]
 [   0   -1]
 [   0    0]]
3298
[[  0 136]
 [  0  45]
 [  0 -17]
 [  0  -8]
 [  0  11]
 [  0   9]
 [  0  -3]
 [  1   3]
 [  0  -2]
 [  0   1]
 [  0  -5]
 [  0   3]
 [  1  -1]
 [  3   3]
 [  0  -7]
 [  0   5]
 [  0   1]
 [  0  -1]
 [  9   1]
 [  0  -1]
 [  0   0]]
3299
[[  0 -41]
 [  0   6]
 [  0 -35]
 [  0  -8]
 [  0   3]
 [  0   1]
 [  1   1]
 [  0   2]
 [  0  -2]
 [  1   2]
 [  0   0]]
cbcr
[[ 0 13]
 [ 0 -6]
 [ 0 14]
 [ 0 -1]
 [ 0 -9]
 [ 0 -1]
 [ 0  1]
 [ 0 -3]
 [ 0 -1]
 [ 0  0]]
[[ 0 12]
 [ 0  7]
 [ 0 -7]
 [ 0  1]
 [ 1 -1]
 [ 1  1]
 [ 0  0]]
3300
[[  0 -65]
 [  0 -24]
 [  0 -12]
 [  0  10]
 [  0  -2]
 [  0   7]
 [  0  -5]
 [  2   1]
 [  0   1]
 [  0   2]
 [  0  -8]
 [  2  -1]
 [  7   1]
 [  0   0]]
3301
[[  0  39]
 [  0  -3]
 [  0 -18]
 [  0   5]
 [  0  -2]
 [  0   3]
 [  2  -2]
 [  0  -2]
 [  0   0]]
3302
[[  0  49]
 [  0 -18]
 [  0 -39]
 [  0  -4]
 [  0  -1]
 [ 

 [  0   0]]
cbcr
[[  0 -12]
 [  0   6]
 [  0  -8]
 [  0   2]
 [  0  -3]
 [  3  -1]
 [  0   0]]
[[ 0  3]
 [ 1  1]
 [ 2  1]
 [ 1 -1]
 [ 0  0]]
3624
[[  0 100]
 [  0  -2]
 [  0  29]
 [  0   1]
 [  0   1]
 [  0  -4]
 [  0  -1]
 [  1  -1]
 [  0   2]
 [  0   0]]
3625
[[ 0  0]
 [ 0 -6]
 [ 0 24]
 [ 0 -1]
 [ 0  2]
 [ 1  1]
 [ 0  1]
 [ 0  1]
 [ 0  2]
 [ 0  0]]
3626
[[  0 -67]
 [  0  -7]
 [  0  30]
 [  0   1]
 [  0   1]
 [  0  -2]
 [  3   2]
 [  0   0]]
3627
[[ 0 12]
 [ 0 -5]
 [ 0 24]
 [ 0 -1]
 [ 1 -3]
 [ 3  1]
 [ 0  0]]
cbcr
[[  0 -35]
 [  0   5]
 [  0  -9]
 [  0   2]
 [  0   2]
 [  0   1]
 [  1   1]
 [  0   1]
 [  0   0]]
[[ 0  3]
 [ 0 -1]
 [ 0  3]
 [ 1 -1]
 [ 0  0]]
3628
[[ 0 53]
 [ 0 12]
 [ 0 21]
 [ 0 -3]
 [ 0 -2]
 [ 0 -1]
 [ 1  1]
 [ 0  1]
 [ 0  1]
 [ 0  0]]
3629
[[  0  -8]
 [  0  -9]
 [  0  32]
 [  0 -13]
 [  0  -4]
 [  0   2]
 [  0   1]
 [  0   1]
 [  0   4]
 [  0   2]
 [  0  -1]
 [  1   2]
 [  0   0]]
3630
[[  0 -50]
 [  0  12]
 [  0  25]
 [  0  -1]
 [  0   3]
 [  0  -3]
 [  3   1]
 [  3 

 [0 0]]
3911
[[ 0 -4]
 [ 0  3]
 [ 0  5]
 [ 0 -1]
 [ 1 -1]
 [ 0  0]]
cbcr
[[ 0  2]
 [ 0 -1]
 [ 0 -2]
 [ 0  0]]
[[0 1]
 [1 1]
 [0 0]]
3912
[[ 0 -6]
 [ 0  6]
 [ 2  1]
 [ 0 -2]
 [ 0  0]]
3913
[[  0 -12]
 [  0   2]
 [  0  -2]
 [  3   1]
 [  2  -1]
 [  0   0]]
3914
[[0 9]
 [0 4]
 [0 2]
 [0 0]]
3915
[[ 0 -7]
 [ 0  3]
 [ 0  1]
 [ 0 -1]
 [ 0  0]]
cbcr
[[ 0 10]
 [ 0 -3]
 [ 2 -1]
 [ 0 -1]
 [ 0  0]]
[[ 0 -2]
 [ 1  2]
 [ 0  0]]
3916
[[  0 -12]
 [  0   6]
 [  0  -4]
 [  0  -1]
 [  1  -2]
 [  0   0]]
3917
[[  0 -21]
 [  0  12]
 [  0  -5]
 [  2  -1]
 [  0   2]
 [  0  -1]
 [  0   0]]
3918
[[ 0 23]
 [ 0  8]
 [ 0  3]
 [ 0  0]]
3919
[[  0 -18]
 [  0   9]
 [  0   4]
 [  0   0]]
cbcr
[[ 0 13]
 [ 0 -5]
 [ 0  1]
 [ 0  1]
 [ 1  1]
 [ 0  0]]
[[0 1]
 [1 1]
 [0 0]]
3920
[[  0 -48]
 [  0  20]
 [  0 -11]
 [  2  -1]
 [  0   2]
 [  2  -1]
 [  0   0]]
3921
[[  0 -37]
 [  0  11]
 [  0 -11]
 [  1   1]
 [  1   1]
 [  2  -1]
 [  0   0]]
3922
[[ 0 60]
 [ 0 14]
 [ 0 -4]
 [ 0 -2]
 [ 0  3]
 [ 0 -1]
 [ 0  1]
 [ 2 -1]
 [ 0  0]]

 [  0   0]]
4277
[[  0  63]
 [  0 -34]
 [  0  11]
 [  0  -1]
 [  0  -2]
 [  1  -3]
 [  2   1]
 [  0   0]]
4278
[[   0 -149]
 [   0   -3]
 [   0   85]
 [   0   22]
 [   0   -4]
 [   0    1]
 [   1    1]
 [   0   -1]
 [   0   -8]
 [   0  -13]
 [   0    1]
 [   7    1]
 [   0   -3]
 [   0    3]
 [  13    2]
 [   0    0]]
4279
[[  0  16]
 [  0 -10]
 [  0 111]
 [  0  30]
 [  0 -11]
 [  0   1]
 [  0  -1]
 [  1  -3]
 [  0  -9]
 [  0 -16]
 [  0   1]
 [  1  -1]
 [  5   2]
 [  0  -4]
 [  0   3]
 [ 13   3]
 [  0   0]]
cbcr
[[  0 -19]
 [  0  10]
 [  0 -26]
 [  0  10]
 [  0   7]
 [  0  -2]
 [  1  -1]
 [  0  -1]
 [  0  -1]
 [  0  -1]
 [  9   1]
 [  0   0]]
[[  0  23]
 [  0 -11]
 [  0  -1]
 [  0   3]
 [  0  -7]
 [  0   1]
 [  1   1]
 [  0   1]
 [  0   0]]
4280
[[  0 194]
 [  0 -16]
 [  0  13]
 [  0  -3]
 [  1  -1]
 [  0  -1]
 [  1  -1]
 [  0   2]
 [  0   0]]
4281
[[ 0 22]
 [ 0 -5]
 [ 0 15]
 [ 0 -1]
 [ 0 -1]
 [ 0 -2]
 [ 3  1]
 [ 0  0]]
4282
[[   0 -196]
 [   0   -6]
 [   0  145]
 [   0   39]
 [   0   

In [158]:
def bit_rep_to_value(bit_rep):
    if bit_rep == '':
        return 0
    sign = 1
    if bit_rep[0]=='0':
        sign = -1
        bit_rep = bit_rep.replace('0', 'a').replace('1','0').replace('a','1')

    return sign*int(bit_rep, 2)
       
    

codes_ac_Y_squished = codes_ac_Y.reshape((1,-1))
codes_ac_C_squished = codes_ac_C.reshape((1,-1))
# print(codes_ac_Y_squished)
# print(codes_ac_C_squished)

def func(data):
    arr = []
    Y = []
    Cb = []
    Cr = []
    count = 0
    while data:
        #dcy ac_y *4 dc_c ac_c*2 
        for i in range(4):
            print(count)
            arr.append(count)
            count+=1
            a = []
            pref = ''
            cur = 0
            while pref not in codes_dc_Y and len(data)>cur:
                pref+=data[cur]
                cur+=1
            else:
                if len(data)<=cur or pref not in codes_dc_Y:
                    print('yo', len(data), cur, pref)
                    print("Fail")
                    return arr
            data = data[cur:]
#             print(pref)
            cat = np.where(codes_dc_Y==pref)[1][0]
            val = bit_rep_to_value(data[:cat])
            data = data[cat:]
            tup = (0, val)
            print(tup)
            a.append(tup)
            
            tup = (-1, -1)
            cnt = 0
            while tup != (0, 0) and data and cnt<63:
                pref = ''
                cur = 0
                while pref not in codes_ac_Y_squished and data:
                    pref+=data[cur]
                    cur+=1
                else:
                    if not data or pref not in codes_ac_Y_squished:
                        print('hi', len(data), cur, pref)
                        print("Fail")
                        return arr
#                 print(pref)
                data = data[cur:]
                ind = np.where(codes_ac_Y_squished==pref)[1][0]
                cat = ind%16
                run = ind//16
                val = bit_rep_to_value(data[:cat])
                data = data[cat:]
                cnt+=(run+1)
                tup = (run, val)
#                 print(tup)
                a.append(tup)
            arr.append(a)
        
        arr.append('cbcr')
        print('cbcr')
        for i in range(2):
            a = []
            pref = ''
            cur = 0
            while pref not in codes_dc_C and len(data)>cur:
                pref+=data[cur]
                cur+=1
            else:
                if len(data)<=cur or pref not in codes_dc_C:
                    print('hello', len(data), cur, pref)
                    print("Fail")
                    return arr
            data = data[cur:]
#             print(pref)
            
            cat = np.where(codes_dc_C==pref)[1][0]
            val = bit_rep_to_value(data[:cat])
            data = data[cat:]
            tup = (0, val)
            print(tup)
            a.append(tup)
            
            tup = (-1, -1)
            cnt=0
            while tup != (0, 0) and data and cnt<63:
                pref = ''
                cur = 0
                while pref not in codes_ac_C_squished and data:
                    pref+=data[cur]
                    cur+=1
                else:
                    if not data or pref not in codes_ac_C_squished:
                        print('yay', len(data), cur, pref)
                        print("Fail")
                        return arr
                data = data[cur:]
                ind = np.where(codes_ac_C_squished==pref)[1][0]
                cat = ind%16
                run = ind//16
                cnt+=(run+1)
                val = bit_rep_to_value(data[:cat])
                data = data[cat:]
                tup = (run, val)
                a.append(tup)
            arr.append(a)
    return arr
        
s = '...'
#bmp
# s = '...'
#my image
# s = '...'
#my image with my code lengths
s = '...'
orig = func(s)

0
(0, 339)
1
(0, 0)
2
(0, 0)
3
(0, 0)
cbcr
(0, 0)
(0, 0)
4
(0, 0)
5
(0, 0)
6
(0, 0)
7
(0, 0)
cbcr
(0, 0)
(0, 0)
8
(0, 0)
9
(0, 0)
10
(0, 0)
11
(0, 0)
cbcr
(0, 0)
(0, 0)
12
(0, 0)
13
(0, 0)
14
(0, 0)
15
(0, 0)
cbcr
(0, 0)
(0, 0)
16
(0, 0)
17
(0, 0)
18
(0, 0)
19
(0, 0)
cbcr
(0, 0)
(0, 0)
20
(0, 0)
21
(0, 0)
22
(0, 0)
23
(0, 0)
cbcr
(0, 0)
(0, 0)
24
(0, 0)
25
(0, 0)
26
(0, 0)
27
(0, 0)
cbcr
(0, 0)
(0, 0)
28
(0, 0)
29
(0, 0)
30
(0, 0)
31
(0, 0)
cbcr
(0, 0)
(0, 0)
32
(0, 0)
33
(0, 0)
34
(0, 0)
35
(0, -21)
cbcr
(0, -2)
(0, 1)
36
(0, 21)
37
(0, -10)
38
(0, -37)
39
(0, -6)
cbcr
(0, -12)
(0, 8)
40
(0, 16)
41
(0, -26)
42
(0, 26)
43
(0, -168)
cbcr
(0, -15)
(0, 10)
44
(0, 122)
45
(0, 54)
46
(0, -63)
47
(0, 62)
cbcr
(0, -9)
(0, -2)
48
(0, 22)
49
(0, 8)
50
(0, -17)
51
(0, 17)
cbcr
(0, 31)
(0, -14)
52
(0, 0)
53
(0, 0)
54
(0, 0)
55
(0, 0)
cbcr
(0, 7)
(0, -3)
56
(0, 0)
57
(0, 0)
58
(0, 0)
59
(0, 0)
cbcr
(0, 0)
(0, 0)
60
(0, 0)
61
(0, 0)
62
(0, 0)
63
(0, 0)
cbcr
(0, 0)
(0, 0)
64
(0, 0)
65
(0, 0)
66
(0, 

708
(0, 0)
709
(0, 0)
710
(0, 0)
711
(0, 0)
cbcr
(0, 0)
(0, 0)
712
(0, 0)
713
(0, 0)
714
(0, 0)
715
(0, 0)
cbcr
(0, 0)
(0, 0)
716
(0, 0)
717
(0, 0)
718
(0, 0)
719
(0, 0)
cbcr
(0, 0)
(0, 0)
720
(0, 0)
721
(0, 0)
722
(0, 0)
723
(0, 0)
cbcr
(0, 0)
(0, 0)
724
(0, 0)
725
(0, 0)
726
(0, 0)
727
(0, 0)
cbcr
(0, 0)
(0, 0)
728
(0, 0)
729
(0, 0)
730
(0, 0)
731
(0, 0)
cbcr
(0, 0)
(0, 0)
732
(0, 0)
733
(0, 0)
734
(0, 0)
735
(0, 0)
cbcr
(0, 0)
(0, 0)
736
(0, 0)
737
(0, 0)
738
(0, 0)
739
(0, 0)
cbcr
(0, 0)
(0, 0)
740
(0, 0)
741
(0, 0)
742
(0, 0)
743
(0, 0)
cbcr
(0, 0)
(0, 0)
744
(0, 0)
745
(0, 0)
746
(0, 0)
747
(0, 0)
cbcr
(0, 0)
(0, 0)
748
(0, 0)
749
(0, 0)
750
(0, 0)
751
(0, 0)
cbcr
(0, 0)
(0, 0)
752
(0, 0)
753
(0, 0)
754
(0, 0)
755
(0, 0)
cbcr
(0, 0)
(0, 0)
756
(0, 0)
757
(0, 0)
758
(0, 0)
759
(0, 0)
cbcr
(0, 0)
(0, 0)
760
(0, 0)
761
(0, 0)
762
(0, 0)
763
(0, 0)
cbcr
(0, 0)
(0, 0)
764
(0, 0)
765
(0, 0)
766
(0, 0)
767
(0, 0)
cbcr
(0, 0)
(0, 0)
768
(0, 0)
769
(0, 0)
770
(0, 0)
771
(0, 0)
cbcr
(0, 0)

In [147]:
orig = [str(ele) for ele in orig]
with open("snail_jpg.txt", 'w') as f:
    f.write('\n'.join(orig))

In [159]:
orig

[0,
 [(0, 339), (0, 0)],
 1,
 [(0, 0), (0, 0)],
 2,
 [(0, 0), (0, 0)],
 3,
 [(0, 0), (0, 0)],
 'cbcr',
 [(0, 0), (0, 0)],
 [(0, 0), (0, 0)],
 4,
 [(0, 0), (0, 0)],
 5,
 [(0, 0), (0, 0)],
 6,
 [(0, 0), (0, 0)],
 7,
 [(0, 0), (0, 0)],
 'cbcr',
 [(0, 0), (0, 0)],
 [(0, 0), (0, 0)],
 8,
 [(0, 0), (0, 0)],
 9,
 [(0, 0), (0, 0)],
 10,
 [(0, 0), (0, 0)],
 11,
 [(0, 0), (0, 0)],
 'cbcr',
 [(0, 0), (0, 0)],
 [(0, 0), (0, 0)],
 12,
 [(0, 0), (0, 0)],
 13,
 [(0, 0), (0, 0)],
 14,
 [(0, 0), (0, 0)],
 15,
 [(0, 0), (0, 0)],
 'cbcr',
 [(0, 0), (0, 0)],
 [(0, 0), (0, 0)],
 16,
 [(0, 0), (0, 0)],
 17,
 [(0, 0), (0, 0)],
 18,
 [(0, 0), (0, 0)],
 19,
 [(0, 0), (0, 0)],
 'cbcr',
 [(0, 0), (0, 0)],
 [(0, 0), (0, 0)],
 20,
 [(0, 0), (0, 0)],
 21,
 [(0, 0), (0, 0)],
 22,
 [(0, 0), (0, 0)],
 23,
 [(0, 0), (0, 0)],
 'cbcr',
 [(0, 0), (0, 0)],
 [(0, 0), (0, 0)],
 24,
 [(0, 0), (0, 0)],
 25,
 [(0, 0), (0, 0)],
 26,
 [(0, 0), (0, 0)],
 27,
 [(0, 0), (0, 0)],
 'cbcr',
 [(0, 0), (0, 0)],
 [(0, 0), (0, 0)],
 28,
 [

In [141]:
# print(C_q)
# print(Cb_blocks[8],'\n',Cb_blocks[9],'\n',Cb_blocks[10])
#Y_rbg, Y_blocks_og, Y_blocks (qt), encoded_Y
# print(encoded_Y[86])
print(Y_blocks_og[86])
cv2.dct(Y_rgb[86])
print(Y_blocks[86])

[[ 3.45250000e+02  4.50119781e+00  7.16123772e+00 -1.23688755e+01
   3.74999976e+00  1.48116112e+01 -2.19081402e+01  7.84162951e+00]
 [ 6.07507812e+02  8.46881104e+01  9.60714161e-01  9.04719543e+00
  -6.81189537e+00 -2.18638763e+01  2.18930855e+01 -5.18625498e+00]
 [-1.22050467e+01 -2.28143509e+02  2.47854903e-01  4.84181732e-01
   4.60580397e+00  2.25497150e+01 -4.79326296e+00 -3.81057763e+00]
 [-1.65991043e+02  2.15779510e+02 -5.54185677e+01  1.20987606e+01
   7.30426216e+00 -1.87703056e+01 -2.29952097e+00  2.70055151e+00]
 [-8.02499847e+01 -4.40141106e+01  1.29472107e+02 -4.07705879e+01
  -1.77500000e+01  7.86109114e+00 -4.92145824e+00  8.38724709e+00]
 [ 2.21954056e+02 -9.97764282e+01 -1.45533798e+02  5.59857941e+01
   1.19179697e+01  7.84184694e+00  1.41201687e+01 -1.87895775e+01]
 [-4.90640907e+01  1.02616814e+02  9.07067413e+01 -4.38950577e+01
   4.58656788e+00 -1.93743267e+01 -1.42478437e+01  2.06080360e+01]
 [-1.02722595e+02 -3.89301605e+01 -2.85225372e+01  1.99329453e+01
  -

In [157]:
file = open('my_image.jpeg', 'rb')
data = file.read()
s = data.hex()

l = re.findall('..?', s)
while len(l) % 11:
    l = np.append(l, '0')
d = np.reshape(l, (-1,11))
d = [' '.join(ele) for ele in d]
print(len(d))
for ele in d:
    print(ele)

file.close()

1137
ff d8 ff e0 00 10 4a 46 49 46 00
01 01 01 00 60 00 60 00 00 ff db
00 43 00 03 02 02 03 02 02 03 03
03 03 04 03 03 04 05 08 05 05 04
04 05 0a 07 07 06 08 0c 0a 0c 0c
0b 0a 0b 0b 0d 0e 12 10 0d 0e 11
0e 0b 0b 10 16 10 11 13 14 15 15
15 0c 0f 17 18 16 14 18 12 14 15
14 ff db 00 43 01 03 04 04 05 04
05 09 05 05 09 14 0d 0b 0d 14 14
14 14 14 14 14 14 14 14 14 14 14
14 14 14 14 14 14 14 14 14 14 14
14 14 14 14 14 14 14 14 14 14 14
14 14 14 14 14 14 14 14 14 14 14
14 14 14 14 ff c0 00 11 08 01 00
01 00 03 01 22 00 02 11 01 03 11
01 ff c4 00 1d 00 01 00 02 03 00
04 00 00 00 00 00 00 00 00 00 00
00 05 06 04 07 08 01 02 03 09 ff
c4 00 42 10 00 02 01 03 03 03 03
01 07 02 02 08 04 08 00 00 01 02
03 00 04 05 06 11 12 07 13 21 08
22 31 14 09 15 23 32 41 51 61 71
81 42 91 16 17 24 33 62 72 82 a1
43 52 b1 c1 25 28 34 53 92 d1 f0
f1 ff c4 00 1c 01 01 00 02 03 01
02 00 00 00 00 00 00 00 00 00 00
00 04 05 02 03 06 01 07 08 ff c4
00 3a 11 00 01 04 00 04 03 05 06
03 07 06 00 00 00 00 00 01 00 02
03 11